In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras
from tensorflow.keras import models
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import to_categorical

import seaborn as sns

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train.head()

In [ ]:
X = train.drop(labels = ['label'], axis=1)
y = train['label']

In [ ]:
# takes a dataframe
# normalizes the pixel values from 0-255 to 0-1
# reshapes the images to 28x28

def preprocess(x):
    x = x / 255.0
    x = x.reshape(-1,28,28,1)
    return x

In [ ]:
# preprocess the test and training data

X = preprocess(X.values)
test_X = preprocess(test.values)

In [ ]:
# one hot encode the labels for 10 classes (one per number 0-9)
y = to_categorical(y)

In [ ]:
# split into training and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=357)

In [ ]:
# creating the model
# dont really know what anything in this section does, just followed a tutorial
model=models.Sequential()
  
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))  

model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
    
model.add(Flatten())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# training the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs = 5, batch_size = 128)

In [ ]:
Y_pred = model.predict(X_val)
pred_classes = np.argmax(Y_pred, axis=1)

In [ ]:
# Generate confusion matrix
Y_true = np.argmax(y_val,axis = 1) 
confusion_mtx = confusion_matrix(Y_true, pred_classes) 


plt.figure(figsize=(10,8))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

sns.heatmap(confusion_mtx, annot=True, fmt="d");

In [ ]:
pred = model.predict(test_X)
classes = np.argmax(pred, axis=1)

In [ ]:
# generate submission file
output = pd.DataFrame({
    'ImageId': [x for x in range(1, len(classes)+1)],
    'Label': classes,
})

output.to_csv("submission.csv", index=False)
output.head()